In [7]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('databases/merged_gtfs.db')
gtfs_names = ['routes', 'stop_times', 'stops', 'transfers', 'trips']

for name in gtfs_names:
    df = pd.read_csv(f"gtfs/{name}.txt")
    df.to_sql(name, conn, if_exists='replace', index=False)

# Close the connection
conn.close()

/tmp/ipykernel_37501/2957384310.py:9: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"gtfs/{name}.txt")


In [9]:
import sqlite3

def test_query(db_path: str, query: str):
    """
    Test function to run a SQL query on the specified SQLite database.

    :param db_path: Path to the SQLite database
    :param query: SQL query to execute
    :return: Results of the query
    """
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Close the connection
        conn.close()

        return results
    except sqlite3.Error as e:
        return {"error": f"SQLite error: {str(e)}"}

# Path to the merged GTFS database
db_path = "merged_gtfs.db"

# Example query to fetch all routes from the 'routes' table
query = "SELECT * FROM routes LIMIT 5;"  # You can modify this query as needed

# Run the query
results = test_query(db_path, query)

# Print the results
if isinstance(results, list):
    for row in results:
        print(row)
else:
    print(results)  # In case of an error, print the error message


('5-T01-y08-1', 1, 'T1', None, 0, 'F47922', '000000')
('11-MGr-n-y08-1', 1, 'MGrn', None, 1, '0AAE53', '000000')
('11-MRe-1-y08-1', 1, 'MRed1', None, 1, 'E91D2F', '000000')
('11-MRe-2-y08-1', 1, 'MRed2', None, 1, 'E91D2F', '000000')
('10', 1, '10', 'Gold Souq Bus Stn - Al Quoz Bus Stn', 3, '6F2E90', 'FFFFFF')


In [4]:
import streamlit as st
from tools import GeocodingTool, GTFSCoordinatorTool, DateTimeTool
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_openai import ChatOpenAI

# Initialize tools
gtfs_tool = GTFSCoordinatorTool()
geocode_tool = GeocodingTool()
dt_tool = DateTimeTool()

# Initialize LLM with system message for proper query format
llm = ChatOpenAI(
                base_url="https://api.sambanova.ai/v1/",
                api_key=st.secrets["SAMBANOVA_API_KEY"], 
                streaming=True,
                model="Meta-Llama-3.1-70B-Instruct",
            )

# Define tools
tools = [
    Tool(
        name="GTFSCoordinatorTool",
        func=gtfs_tool.run,
        description="Handles queries related to public transit data such as schedules, routes, and stop information"
    ),
    Tool(
        name="GeocodingTool",
        func=geocode_tool.run,
        description="Fetches geocoding information from TomTom's API based on user inputs"
    ),
    Tool(
        name="DateTimeTool", 
        func=dt_tool.run,
        description="Handles various date and time operations"
    )
]

# Initialize agent with tools and system message
agent = initialize_agent(
    tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# Test query
query = "Nearest bus stops to Dubai Mall and when does the next bus arrive?"
response = agent.run(query)
print("Response: ", response)




> Entering new AgentExecutor chain...
To find the nearest bus stops to Dubai Mall and the arrival time of the next bus, we first need to determine the location of Dubai Mall in a format that can be used by the public transit system. This involves geocoding the location of Dubai Mall.

Action: GeocodingTool
Action Input: "Dubai Mall"Request URL: https://api.tomtom.com/search/2/geocode/Dubai Mall.json
Response Status: 200

Observation: {'summary': {'query': 'dubai mall', 'queryType': 'NON_NEAR', 'queryTime': 42, 'numResults': 1, 'offset': 0, 'totalResults': 1, 'fuzzyLevel': 1}, 'results': [{'type': 'Street', 'id': 'hTZZWXwiBDcnEF95atzRVA', 'score': 4.5216622353, 'matchConfidence': {'score': 0.9673646368290532}, 'address': {'streetName': 'Dubai Mall Crescent', 'municipalitySubdivision': 'Zaabeel', 'municipality': 'Dubai', 'municipalitySecondarySubdivision': 'Burj Khalifa', 'countrySubdivision': 'Dubai', 'countrySubdivisionName': 'Dubai', 'countrySubdivisionCode': 'DU', 'countryCode': 'A